<a href="https://colab.research.google.com/github/orifelszer/CrimeData/blob/eden-branch/DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [2]:
# ✅ שלב 1: שכפול המאגר מה-GitHub
!git clone https://github.com/orifelszer/CrimeData.git

# ✅ שלב 2: פונקציה לפתיחת קובצי ZIP מהתיקייה המקומית
def load_zipped_csv_local(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        csv_name = zip_ref.namelist()[0]
        with zip_ref.open(csv_name) as file:
            return pd.read_csv(file)

# ✅ שלב 3: קריאת הנתונים לאחר השכפול (נתיב מקומי)
X_train = load_zipped_csv_local('CrimeData/X_train_supervised.zip')
X_test = load_zipped_csv_local('CrimeData/X_test_supervised.zip')
y_train = load_zipped_csv_local('CrimeData/y_train_supervised.zip')
y_test = load_zipped_csv_local('CrimeData/y_test_supervised.zip')

# ✅ בדיקה שהנתונים נטענו כראוי
print(f"X_train Shape: {X_train.shape}")
print(f"X_test Shape: {X_test.shape}")
print(f"y_train Shape: {y_train.shape}")
print(f"y_train Shape: {y_train.shape}")

Cloning into 'CrimeData'...
remote: Enumerating objects: 356, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 356 (delta 95), reused 18 (delta 18), pack-reused 188 (from 2)
Receiving objects: 100% (356/356), 211.20 MiB | 22.97 MiB/s, done.
Resolving deltas: 100% (178/178), done.
X_train Shape: (1257198, 10)
X_test Shape: (376971, 10)
y_train Shape: (1257198, 1)
y_train Shape: (1257198, 1)


In [ ]:
from tensorflow.keras.models import Model

# הוספת EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# בניית המודל מחדש
model = Sequential()
model.add(Dense(512, kernel_regularizer=l2(0.01), input_shape=(X_train.shape[1],)))
model.add(LeakyReLU(alpha=0.01))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(256, kernel_regularizer=l2(0.01)))
model.add(LeakyReLU(alpha=0.01))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(128, kernel_regularizer=l2(0.01)))
model.add(LeakyReLU(alpha=0.01))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(y_train['target'].nunique(), activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# אימון המודל כולל EarlyStopping
history = model.fit(X_train, y_train['target'],
                    epochs=50, batch_size=512,
                    validation_split=0.2,
                    callbacks=[early_stopping, reduce_lr])

Epoch 1/50
1965/1965 ━━━━━━━━━━━━━━━━━━━━ 75s 36ms/step - accuracy: 0.3215 - loss: 2.8257 - val_accuracy: 0.3802 - val_loss: 1.5850 - learning_rate: 0.0010
Epoch 2/50
 587/1965 ━━━━━━━━━━━━━━━━━━━━ 45s 33ms/step - accuracy: 0.3663 - loss: 1.6086

In [23]:
# הערכת ביצועים על סט הבדיקה
test_loss, test_accuracy = model.evaluate(X_test, y_test['target'])
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

ValueError: Layer "functional_26" expects 4 input(s), but it received 1 input tensors. Inputs received: [<tf.Tensor 'IteratorGetNext:0' shape=(None, 10) dtype=float32>]